In [1]:
# imports
import zipfile
import base64
import io
import glob
import time
import json
import os
import requests
import mimetypes
import csv
import pandas as pd
import numpy as np

from pypdf import PdfReader
from base64 import b64decode

from bs4 import BeautifulSoup

In [117]:
import re

[Getting an absolute path from an interactive shell](https://bobbyhadz.com/blog/python-nameerror-name-file-is-not-defined)

In [44]:
df = pd.read_csv('anti-lgbtq-bills-abbrevs-docids-billinfo-tracker.csv', usecols=['State','Number','Bill ID','Bill Text','Bill Text Info'])

In [45]:
df.head()

,State,Number,Bill ID,Bill Text,Bill Text Info
0,Alaska,HB27,1646385,NaN,"{'doc_id': 2626455, 'bill_id': 1646385, 'date'..."
1,Arizona,HB2312,1657170,NaN,"{'doc_id': 2638623, 'bill_id': 1657170, 'date'..."
2,Arizona,HB2711,1696919,NaN,"{'doc_id': 2684031, 'bill_id': 1696919, 'date'..."
3,Arizona,SB1001,1639597,NaN,"{'doc_id': 2619652, 'bill_id': 1639597, 'date'..."
4,Arizona,SB1026,1644653,NaN,"{'doc_id': 2624872, 'bill_id': 1644653, 'date'..."


In [46]:
df['mime'] = ''
df.head()

,State,Number,Bill ID,Bill Text,Bill Text Info,mime
0,Alaska,HB27,1646385,NaN,"{'doc_id': 2626455, 'bill_id': 1646385, 'date'...",
1,Arizona,HB2312,1657170,NaN,"{'doc_id': 2638623, 'bill_id': 1657170, 'date'...",
2,Arizona,HB2711,1696919,NaN,"{'doc_id': 2684031, 'bill_id': 1696919, 'date'...",
3,Arizona,SB1001,1639597,NaN,"{'doc_id': 2619652, 'bill_id': 1639597, 'date'...",
4,Arizona,SB1026,1644653,NaN,"{'doc_id': 2624872, 'bill_id': 1644653, 'date'...",


In [28]:
import ast

In [47]:
def billtextinfo_string_to_dict(index):
    try:
        result = ast.literal_eval(index)
        return result
    except:
        return None

In [48]:
df['Bill Text Info'] = df.apply(lambda row: billtextinfo_string_to_dict(row['Bill Text Info']), axis=1)

In [58]:
df['Bill Text Info'][1]['mime']

'text/html'

In [55]:
def get_mime(index):
    try:
        return index['mime']
    except: return None
df['mime'] = df.apply(lambda row: get_mime(row['mime']), axis=1)

In [59]:
billtextinfodicts = list(df['Bill Text Info'])

In [60]:
type(billtextinfodicts[0])

dict

In [71]:
billtextinfodicts[0]['mime']

'application/pdf'

In [ ]:
# turn the bills with mime application/pdf from base64 into PDFs
for billtextinfo in billtextinfodicts:
    try:        
        if(billtextinfo['mime'] == 'application/pdf'):
            b64 = billtextinfo['doc']
            bytes = b64decode(b64, validate=True)

            # Perform a basic validation to make sure that the result is a valid PDF file
            # Be aware! The magic number (file signature) is not 100% reliable solution to validate PDF files
            # Moreover, if you get Base64 from an untrusted source, you must sanitize the PDF contents
            if bytes[0:4] != b'%PDF':
              raise ValueError('Missing the PDF file signature')

            bill_id_name = billtextinfo['bill_id']

            # Write the PDF contents to a local file
            f = open(f"bill_id-{bill_id_name}.pdf", "wb")
            f.write(bytes)
            f.close()
        else:
            continue
    except AttributeError:
        print("NoneType")
# this works; just the last one throws the error

In [ ]:
billtextinfodicts[0]['doc']

In [ ]:
# turn the bills with mime text/html from base64 into HTML
for billtextinfo in billtextinfodicts:
    try:        
        if(billtextinfo['mime'] == 'text/html'):
            b64 = billtextinfo['doc']
            bytes = b64decode(b64, validate=True)

            bill_id_name = billtextinfo['bill_id']

            # Write the PDF contents to a local file
            f = open(f"bill_id-{bill_id_name}.html", "wb")
            f.write(bytes)
            f.close()

            with open(f"bill_id-{bill_id_name}.html") as fp:
                soup = BeautifulSoup(fp.decode('utf-8','ignore'))
                i = open(f"bill_id-{bill_id_name}.txt", "w")
                i.write(soup.get_text())
                i.close()    
            
        else:
            continue
    except AttributeError:
        print("NoneType")
# this produced the HTML files but not the txt files

In [146]:
import codecs

In [147]:
# turn the HTML files into txts
filedir = f"/Users/gabriel/Documents/GitHub/legiscan/azcentral-text-reuse-model-legislation/notebooks/20230302 Bill HTMLs/"
for file in os.listdir(filedir):
    if file.endswith('.html'):
        try:
            billnumber = re.search(r"\d{7}", file)
            filename = billnumber.group()
            with codecs.open(f"{filedir}{file}", encoding="utf-8", errors="ignore") as fp:
                soup = BeautifulSoup(fp)
                i = open(f"{filedir}bill_id-{filename}.txt", "w")
                i.write(soup.get_text())
                i.close()    

        except AttributeError:
            print("NoneType")

In [114]:
for billtextinfo in billtextinfodicts:
    print(type(billtextinfo))
    print(billtextinfo['mime'])

<class 'dict'>
application/pdf
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
text/html
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
application/pdf
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<class 'dict'>
text/html
<clas

TypeError: 'NoneType' object is not subscriptable

In [127]:
# success
filepath = f"/Users/gabriel/Documents/GitHub/legiscan/azcentral-text-reuse-model-legislation/notebooks/2023-03-02 Bill PDFs/bill_id-1555547.pdf"
billnumber = re.search(r"\d{7}", filepath)
filename = f"bill_id-{billnumber.group()}"
filedir = filepath.replace(f"{filename}.pdf","")
print(filedir)

/Users/gabriel/Documents/GitHub/legiscan/azcentral-text-reuse-model-legislation/notebooks/2023-03-02 Bill PDFs/


In [129]:
# try parsing just one pdf
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
import tika
from tika import parser
import time
parsed_pdf = parser.from_file(filepath) 
txt_data = parsed_pdf['content']
file_out = f"{filedir}{filename}.txt"
with open(file_out, 'w') as outfile:
    outfile.write(txt_data)
# success

In [130]:
pdfs = []
# get the list of files to parse
for files in os.listdir(filedir):
    if files.endswith('.pdf'):
        pdfs.append(files)

In [138]:
for file in os.listdir(filedir):
    if file.endswith('.pdf'):
        parsed_pdf = parser.from_file(filedir+file)
        txt_data = parsed_pdf['content']
        billnumber = re.search(r"\d{7}", file)
        filename = f"bill_id-{billnumber.group()}"
        file_out = f"{filedir}{filename}.txt"
        with open(file_out, 'w') as outfile:
            outfile.write(txt_data)
# success!

In [ ]:
# code from jon
# you only need this if there is a timeout error when you first run this
os.environ['TIKA_SERVER_JAR'] = 'https://repo1.maven.org/maven2/org/apache/tika/tika-server/1.19/tika-server-1.19.jar'
import tika
from tika import parser
import time
one_pdf = ["bill_id-1639439.pdf"]
for urls in one_pdf:
    time.sleep(2)
    file_n = urls.split('/')[-1]
    print(file_n)
    file_name = "/Users/thirkield/Documents/MajorStudio2023/Gabriel_python/" + file_n
    parsed_pdf = parser.from_file(file_name) 
    txt_data = parsed_pdf['content']
    txt_name = file_n.split('.')[0] + "NEW.txt"
    print(txt_name)
    file_out ="/Users/thirkield/Documents/MajorStudio2023/Gabriel_python/" + txt_name
    with open(file_out, 'w') as outfile:
        outfile.write(txt_data)